# Not your father's email: Modern APIs for transactional and mass mailing

Eddie Cosma | CLEPy Meetup | January 13, 2025


## Goals

- Review use cases for mail send APIs
- Compare popular solutions (e.g., SendGrid, MailGun, Amazon SES)
- Demonstrate implementation in Python


# History of email

- Been around since the '60s
- Everyone has it
- Open standards (SMTP, IMAP)


# Comparing methods of notification

- Email still has its place!
- Compared to SMS or Push:
  - Better for long-form content
  - Less intrusive/urgent
  - Accessible from any device

![](img/comparison.png)

Source: https://www.mgt-commerce.com/blog/magento-email-vs-sms-vs-push-notifications/

# The old way

![](img/old-way.png)

Source: https://stackoverflow.com/questions/161212/sending-mass-emails-programmatically

## Example plaintext email

In [52]:
# Get our email password and API keys for demo
with open('.secrets', 'r', encoding='utf-8') as f:
    gmail_password = f.readline()[:-1]
    sendgrid_api_key = f.readline()[:-1]
    sendgrid_template_id = f.readline()[:-1]
    to_email = f.readline()[:-1]
    from_email = f.readline()[:-1]

In [14]:
# Setup our email
from email.message import EmailMessage

message = EmailMessage()
message['From'] = 'clepydemo@gmail.com'
message['To'] = to_email
message['Subject'] = 'Hello world!'

content = """
This is an example of how you can use SMTP to send emails with Python.
"""

message.set_content(content)

# Review our message
print(message.as_string())

From: clepydemo@gmail.com
To: eddie.cosma@gmail.com
Subject: Hello world!
Content-Type: text/plain; charset="utf-8"
Content-Transfer-Encoding: 7bit
MIME-Version: 1.0


This is an example of how you can use SMTP to send emails with Python.



In [9]:
# Send our message
import smtplib

with smtplib.SMTP('smtp.gmail.com', 587) as s:
    s.ehlo()
    s.starttls()
    s.ehlo()
    s.login('clepydemo@gmail.com', gmail_password)
    s.send_message(message)

## HTML emails

```python
from email.utils import make_msgid
email_cid = make_msgid()
message.add_alternative("""\
<html>
  <head></head>
  <body>
    <p>Salut!</p>
    <p>Cela ressemble à un excellent
        <a href="http://www.yummly.com/recipe/Roasted-Asparagus-Epicurious-203718">
            recipie
        </a> déjeuner.
    </p>
    <img src="cid:{email_cid}" />
  </body>
</html>
""".format(email_cid=email_cid[1:-1]), subtype='html')
```

Source: https://docs.python.org/3.12/library/email.examples.html

# Problems with programmatic SMTP send

- Lots of boilerplate
- Requires writing HTML and managing templates (e.g., with Jinja)
  - Email HTML is like dealing with IE 15 years ago
- Manual checks for bounces
- Manual management of unsubscribes
- Mail host required

# Benefits of email APIs

- Replaced SMTP-based email with calls to web APIs
- Allow for advanced insight into outgoing mail
  - "Deliverability" information (e.g., delivered, opened, clicked, bounced, spam)
  - Managed unsubscribe groups
- Template-based
  - Create HTML template with service host
  - Transmit only data that changes as, e.g., JSON
  - Avoid creating RFC 5322 messages locally

# Comparison of services

|                              | SendGrid                                    | Mailgun                                       | Amazon SES                           | Postmark                                 |
|------------------------------|---------------------------------------------|-----------------------------------------------|--------------------------------------|------------------------------------------|
| **Free Tier**                | 100 emails/day (no time limit)              | 100 emails/day (no time limit)                | 3,000 emails/month (12 months only)  | 100 emails/**month** (no time limit)     |
| **Pricing Beyond Free Tier** | Starting at \$19.95/month for 50,000 emails | Starting at \$15/month for 10,000 emails      | Pay-as-you-go at \$0.10/1,000 emails | Starting at \$15/month for 10,000 emails |
| **Template support**         | Yes, JSON transmission                      | Yes, JSON transmission                        | Yes, JSON transmission               | Yes, JSON transmission                   |

## Summary

- API services are very comparable
- If \<100 emails/month, use SendGrid or MailGun
- If \>100, consider Amazon SES
- Evaluate any other unique needs individually



# SendGrid using `requests`

In [53]:
# Setup our environment
import requests

base_url = 'https://api.sendgrid.com'
endpoint = '/v3/mail/send'

In [54]:
# Send a plaintext email
headers = {'Authorization': 'Bearer ' + sendgrid_api_key}
body = {
    'personalizations': [{'to': [{'email': to_email}]}],
    'from': {'email': from_email},
    'subject': 'Hello world!',
    'content': [
        {
            'type': 'text/plain',
            'value': 'This is an example of how you can use SendGrid to send emails with Python.',
        }
    ]
}
response = requests.post(base_url + endpoint, headers=headers, json=body)

print(f'{response.status_code=}')
print(f'{response.text=}')

response.status_code=202
response.text=''


# SendGrid using the sendgrid library

Installation: `pip install sendgrid`

```python3
import sendgrid
from sendgrid.helpers.mail import Mail

sg = sendgrid.SendGridAPIClient(api_key=sendgrid_api_key)
data = {
    'personalizations': [{'to': [{'email': to_email}]}],
    'from': {'email': from_email},
    'subject': 'Hello world!',
    'content': [
        {
            'type': 'text/plain',
            'value': 'This is an example of how you can use SendGrid to send emails with Python.',
        }
    ]
}
response = sg.client.mail.send.post(request_body=data)
```

# SendGrid using the mail helper library

In [35]:
import sendgrid
from sendgrid.helpers.mail import Mail

In [33]:
# Define a function to send a given input message

def send_email(message: Mail) -> None:
    # Set up the API client
    sg = sendgrid.SendGridAPIClient(api_key=sendgrid_api_key)

    # Attempt to send the message
    response = sg.send(message)

    # Show the response
    print(f'{response.status_code=}')
    print(f'{response.to_dict=}')

In [31]:
# Send a plaintext email

message = Mail(
    from_email=from_email,
    to_emails=to_email,
    subject='Hello world!',
    plain_text_content='This is an example of how you can use the SendGrid **library** to send emails with Python.',
)
send_email(message)

response.status_code=202
response.body=b''
response.headers=<http.client.HTTPMessage object at 0x10ed77fb0>


In [ ]:
# Send an HTML email

message = Mail(
    from_email=from_email,
    to_emails=to_email,
    subject='Hello world!',
    html_content='This is an example of how you can use the SendGrid library to send <span style="font-weight: bold;">HTML</span> emails with Python.',
)
send_email(message)

# Using transactional templates

![](img/template-1.png)

![](img/template-2.png)

![](img/template-3.png)

In [37]:
message = Mail(
    from_email=from_email,
    to_emails=to_email,
)
message.dynamic_template_data = {
    'confirm_uri': 'https://www.example.com'
}
message.template_id = sendgrid_template_id
send_email(message)

response.status_code=202
response.body=b''


# Other features

A "kitchen sink" example is provided in the library documentation: https://github.com/sendgrid/sendgrid-python/blob/main/use_cases/kitchen_sink.md

## Scheduled send

```python3
from datetime import datetime, timezone
from sendgrid.helpers.mail import SendAt

dt = datetime(2025, 2, 1, 12, 0, 0)
timestamp = int(dt.replace(tzinfo=timezone.utc).timestamp())

message.send_at = SendAt(timestamp)
```

## Batched send
```python3
from sendgrid.helpers.mail import BatchId

message.batch_id = BatchId("HkJ5yLYULb7Rj8GKSx7u025ouWVlMgAi")
```

## Suppression groups

![](img/unsubscribe.png)

```python3
from sendgrid.helpers.mail import Asm, GroupId, GroupsToDisplay

message.asm = Asm(GroupId(86397), GroupsToDisplay([86397, 12345]))
```




# Other endpoints

See: https://github.com/sendgrid/sendgrid-python/blob/main/USAGE.md

- ACCESS SETTINGS
- ALERTS
- API KEYS
- ASM
- BROWSERS
- CAMPAIGNS
- CATEGORIES
- CLIENTS
- CONTACTDB
- DEVICES
- GEO
- IPS
- MAIL
- MAIL SETTINGS
- MAILBOX PROVIDERS
- PARTNER SETTINGS
- SCOPES
- SENDERS
- SENDER AUTHENTICATION
- STATS
- SUBUSERS
- SUPPRESSION
- TEMPLATES
- TRACKING SETTINGS
- USER

## Example: retrieve suppression group information

In [48]:
sg = sendgrid.SendGridAPIClient(api_key=sendgrid_api_key)

params = {'id': 86397}
response = sg.client.asm.groups.get(query_params=params)

print(f'{response.status_code=}')
print(f'{response.to_dict=}')

response.status_code=200
response.to_dict=[{'name': 'Medcopia Basic', 'id': 86397, 'description': 'Basic alerts for new and ended drug shortages', 'is_default': True, 'unsubscribes': 13}]


# Example: retrieve global email statistics

In [51]:
params = {'aggregated_by': 'day', 'limit': 1, 'start_date': '2025-01-05', 'end_date': '2025-01-05'}
response = sg.client.stats.get(query_params=params)

print(f'{response.status_code=}')
print(f'{response.to_dict=}')

response.status_code=200
response.to_dict=[{'date': '2025-01-05', 'stats': [{'metrics': {'blocks': 1, 'bounce_drops': 0, 'bounces': 1, 'clicks': 0, 'deferred': 0, 'delivered': 55, 'invalid_emails': 0, 'opens': 0, 'processed': 57, 'requests': 57, 'spam_report_drops': 0, 'spam_reports': 0, 'unique_clicks': 0, 'unique_opens': 0, 'unsubscribe_drops': 0, 'unsubscribes': 0}}]}]
